## Import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import os
import argparse
from subprocess import call


import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
tqdm.pandas()

from IPython.display import display, Markdown, Latex, HTML

import torch

from transformers import GPT2Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer


from verisci.covid import AbstractRetriever, RationaleSelector, LabelPredictor
from verisci.evaluate.lib.data import GoldDataset
#import wandb

/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
loc_df_scispacy_sentence_word_unq_ner_abr_filtered ='../../dfs_generated/linguistic/df_scispacy_sentence_word_unq_ner_abr_filtered.pkl'

## Scifact

### Load Data

In [4]:
loc_corpus = '../../data/scifact/corpus.jsonl'
loc_claim_train = '../../data/scifact/claims_train.jsonl'
loc_claim_test = '../../data/scifact/claims_test.jsonl'
loc_claim_dev = '../../data/scifact/claims_dev.jsonl'

In [5]:
from verisci.evaluate.lib.data import GoldDataset

In [6]:
def get_claim_label_from_jsonl(dataset_jsonl):
    claim_label_list_train = []


    for cur_claim in dataset_jsonl:
        claim_txt = cur_claim.claim

        for doc_id, evidence in cur_claim.evidence.items():

            ev_doc = cur_claim.release.corpus.get_document(doc_id)

            claim_label = evidence.label.name

            tmp_dic = {"claim" : claim_txt, "label" : claim_label}

            claim_label_list_train.append(tmp_dic)
    return claim_label_list_train

In [7]:
def get_claim_label_evidence_from_jsonl(dataset_jsonl, source):
    claim_label_list_train = []


    for cur_claim in dataset_jsonl:
        claim_txt = cur_claim.claim

        for doc_id, evidence in cur_claim.evidence.items():

            ev_doc = cur_claim.release.corpus.get_document(doc_id)

            claim_label = evidence.label.name
            
            list_rationales = []
            for i, sents in enumerate(evidence.rationales):
                list_rationales = [sent for i, sent in enumerate(ev_doc.sentences) if i in sents]

            tmp_dic = {"claim" : claim_txt, "label" : claim_label, "list_rationales" :list_rationales, "source" :source}

            claim_label_list_train.append(tmp_dic)
    return claim_label_list_train

In [8]:
ds_train = GoldDataset(loc_corpus, loc_claim_train)
# claim_train = ds_train.get_claim(39)
# claim_train.pretty_print()
dic_train = get_claim_label_evidence_from_jsonl(ds_train, source = "train")

In [9]:
ds_valid = GoldDataset(loc_corpus, loc_claim_dev)
# claim_valid = ds_valid.get_claim(42)
# claim_valid.pretty_print()
dic_valid = get_claim_label_evidence_from_jsonl(ds_valid, source = "dev")

In [10]:
df_claim_evid_label = pd.concat([pd.DataFrame(dic_train), pd.DataFrame(dic_valid)], ignore_index=True)

df_claim_evid_label

,claim,label,list_rationales,source
0,1 in 5 million in UK have abnormal PrP positiv...,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train
1,32% of liver transplantation programs required...,SUPPORTS,[Policies requiring discontinuation of methado...,train
2,40mg/day dosage of folic acid and 2mg/day dosa...,SUPPORTS,[CONCLUSION Treatment with high doses of folic...,train
3,76-85% of people with severe mental disorder r...,SUPPORTS,[Although disorder severity was correlated wit...,train
4,A T helper 2 cell (Th2) environment impedes di...,REFUTES,"[Thus, in Lyn(-/-) mice, basophils and IgE aut...",train
...,...,...,...,...
768,Women with a higher birth weight are more like...,SUPPORTS,[Increased risk of breast cancer was noted wit...,dev
769,Women with a higher birth weight are more like...,SUPPORTS,[RESULTS We found that heavier birth weights w...,dev
770,aPKCz causes tumour enhancement by affecting g...,REFUTES,"[Taken together, this demonstrates that PKCζ i...",dev
771,cSMAC formation enhances weak ligand signalling.,SUPPORTS,[This conclusion was supported by experiments ...,dev


## Abbreviations

In [11]:
import spacy

# nlp = spacy.load("en_core_web_trf")

from spacy import displacy
from nltk import word_tokenize
import nltk

In [12]:
nlp_sci_brt = spacy.load("en_core_sci_scibert")
nlp_sci_lg = spacy.load("en_core_sci_lg")

#import en_ner_bc5cdr_md
nlp_cr = spacy.load("en_ner_craft_md")
nlp_bc = spacy.load("en_ner_bc5cdr_md")
nlp_bi = spacy.load("en_ner_bionlp13cg_md")
nlp_jn = spacy.load("en_ner_jnlpba_md")

nlp_core_web_trf = spacy.load("en_core_web_trf")

In [13]:
from scispacy.abbreviation import AbbreviationDetector

In [14]:
nlp_sci_brt.add_pipe("abbreviation_detector")
nlp_sci_lg.add_pipe("abbreviation_detector")
nlp_cr.add_pipe("abbreviation_detector")
nlp_bc.add_pipe("abbreviation_detector")
nlp_bi.add_pipe("abbreviation_detector")
nlp_jn.add_pipe("abbreviation_detector")

In [31]:
list_sci_brt = set()
list_sci_lg  = set()
list_cr = set()
list_bc = set()
list_bi = set()
list_jn = set()


In [25]:
list_jn.difference(list_jn)

set()

In [36]:
list_jn.difference(list_sci_brt)

set()

In [37]:
list_jn.difference(list_sci_brt.union(list_sci_lg, list_cr, list_bc, list_bi))

set()

In [33]:
all_sentence_abbr = []
for cur_indx, cur_row in tqdm(df_claim_evid_label.iterrows(), total = len(df_claim_evid_label["claim"])):
    
    cur_claim = cur_row['claim']
    doc_sci_brt = nlp_sci_brt(cur_claim)
    
    doc_sci_lg = nlp_sci_lg(cur_claim)
    
    doc_cr = nlp_cr(cur_claim)
    
    
    doc_bc = nlp_bc(cur_claim)
    
    doc_bi = nlp_bi(cur_claim)
    
    doc_jn = nlp_jn(cur_claim)
    
    
    
    
    
    cur_all_abbr = {}
    
    for cur_abr in doc_sci_brt._.abbreviations:  
        list_sci_brt.add(str(cur_abr))
        
        cur_all_abbr[str(cur_abr)] = {'abrv_text': str(cur_abr), 'abr_definition' : str(cur_abr._.long_form), 
                                      'abr_model' : 'en_core_sci_scibert', 'claim' : cur_claim, 'sf_start_char' : cur_abr.start_char, 'sf_end_char' : cur_abr.end_char,
                                     'lf_start_char' : cur_abr._.long_form.start_char, 'lf_end_char' : cur_abr._.long_form.end_char, 
                                     'org_label' : cur_row['label'], 'list_rationales' : cur_row['list_rationales'],
                                      'data_source' : cur_row['source']}
        cur_all_abbr[str(cur_abr)]['abr_whole_start'] = -1
        cur_all_abbr[str(cur_abr)]['abr_whole_end'] = -1
        # if short and long form of abbraviation exists
        if((cur_abr.end_char - cur_abr.start_char > 0) & (cur_abr._.long_form.end_char - cur_abr._.long_form.start_char > 0)):
            # Aaa Bbb Rrr (ABR) or ABR (Aaa Bbb Rrr)  
            if (cur_abr.start_char - cur_abr._.long_form.end_char   < 4 )| (cur_abr._.long_form.start_char - cur_abr.end_char < 4):
                cur_all_abbr[str(cur_abr)]['abr_whole_start'] = min(cur_abr.start_char, cur_abr._.long_form.start_char)
                cur_all_abbr[str(cur_abr)]['abr_whole_end'] = max(cur_abr.end_char, cur_abr._.long_form.end_char)
    
    for cur_abr in doc_sci_lg._.abbreviations:  
        list_sci_lg.add(str(cur_abr))
        cur_all_abbr[str(cur_abr)] = {'abrv_text': str(cur_abr), 'abr_definition' : str(cur_abr._.long_form), 
                                      'abr_model' : 'en_core_sci_lg', 'claim' : cur_claim, 'sf_start_char' : cur_abr.start_char, 'sf_end_char' : cur_abr.end_char,
                                     'lf_start_char' : cur_abr._.long_form.start_char, 'lf_end_char' : cur_abr._.long_form.end_char, 
                                     'org_label' : cur_row['label'], 'list_rationales' : cur_row['list_rationales'],
                                      'data_source' : cur_row['source']}
        
        cur_all_abbr[str(cur_abr)]['abr_whole_start'] = -1
        cur_all_abbr[str(cur_abr)]['abr_whole_end'] = -1
        # if short and long form of abbraviation exists
        if((cur_abr.end_char - cur_abr.start_char > 0) & (cur_abr._.long_form.end_char - cur_abr._.long_form.start_char > 0)):
            # Aaa Bbb Rrr (ABR) or ABR (Aaa Bbb Rrr)  
            if (cur_abr.start_char - cur_abr._.long_form.end_char   < 4 )| (cur_abr._.long_form.start_char - cur_abr.end_char < 4):
                cur_all_abbr[str(cur_abr)]['abr_whole_start'] = min(cur_abr.start_char, cur_abr._.long_form.start_char)
                cur_all_abbr[str(cur_abr)]['abr_whole_end'] = max(cur_abr.end_char, cur_abr._.long_form.end_char)
                
                
    for cur_abr in doc_cr._.abbreviations:  
        list_cr.add(str(cur_abr))
        cur_all_abbr[str(cur_abr)] = {'abrv_text': str(cur_abr), 'abr_definition' : str(cur_abr._.long_form), 
                                      'abr_model' : 'en_ner_craft_md', 'claim' : cur_claim, 'sf_start_char' : cur_abr.start_char, 'sf_end_char' : cur_abr.end_char,
                                      'lf_start_char' : cur_abr._.long_form.start_char, 'lf_end_char' : cur_abr._.long_form.end_char, 
                                      'org_label' : cur_row['label'], 'list_rationales' : cur_row['list_rationales'],
                                      'data_source' : cur_row['source']}
    
    
        cur_all_abbr[str(cur_abr)]['abr_whole_start'] = -1
        cur_all_abbr[str(cur_abr)]['abr_whole_end'] = -1
        # if short and long form of abbraviation exists
        if((cur_abr.end_char - cur_abr.start_char > 0) & (cur_abr._.long_form.end_char - cur_abr._.long_form.start_char > 0)):
            # Aaa Bbb Rrr (ABR) or ABR (Aaa Bbb Rrr)  
            if (cur_abr.start_char - cur_abr._.long_form.end_char   < 4 )| (cur_abr._.long_form.start_char - cur_abr.end_char < 4):
                cur_all_abbr[str(cur_abr)]['abr_whole_start'] = min(cur_abr.start_char, cur_abr._.long_form.start_char)
                cur_all_abbr[str(cur_abr)]['abr_whole_end'] = max(cur_abr.end_char, cur_abr._.long_form.end_char)
                
                
    for cur_abr in doc_bc._.abbreviations:
        list_bc.add(str(cur_abr))
        cur_all_abbr[str(cur_abr)] = {'abrv_text': str(cur_abr), 'abr_definition' : str(cur_abr._.long_form), 
                                      'abr_model' : 'en_ner_bc5cdr_md', 'claim' : cur_claim, 'sf_start_char' : cur_abr.start_char, 'sf_end_char' : cur_abr.end_char,
                                     'lf_start_char' : cur_abr._.long_form.start_char, 'lf_end_char' : cur_abr._.long_form.end_char, 
                                     'org_label' : cur_row['label'], 'list_rationales' : cur_row['list_rationales'],
                                      'data_source' : cur_row['source']}
        
        
        cur_all_abbr[str(cur_abr)]['abr_whole_start'] = -1
        cur_all_abbr[str(cur_abr)]['abr_whole_end'] = -1
        # if short and long form of abbraviation exists
        if((cur_abr.end_char - cur_abr.start_char > 0) & (cur_abr._.long_form.end_char - cur_abr._.long_form.start_char > 0)):
            # Aaa Bbb Rrr (ABR) or ABR (Aaa Bbb Rrr)  
            if (cur_abr.start_char - cur_abr._.long_form.end_char   < 4 )| (cur_abr._.long_form.start_char - cur_abr.end_char < 4):
                cur_all_abbr[str(cur_abr)]['abr_whole_start'] = min(cur_abr.start_char, cur_abr._.long_form.start_char)
                cur_all_abbr[str(cur_abr)]['abr_whole_end'] = max(cur_abr.end_char, cur_abr._.long_form.end_char)
                
                
    for cur_abr in doc_bi._.abbreviations:
        list_bi.add(str(cur_abr))
        cur_all_abbr[str(cur_abr)] = {'abrv_text': str(cur_abr), 'abr_definition' : str(cur_abr._.long_form), 
                                      'abr_model' : 'en_ner_bionlp13cg_md', 'claim' : cur_claim, 'sf_start_char' : cur_abr.start_char, 'sf_end_char' : cur_abr.end_char,
                                     'lf_start_char' : cur_abr._.long_form.start_char, 'lf_end_char' : cur_abr._.long_form.end_char, 
                                     'org_label' : cur_row['label'], 'list_rationales' : cur_row['list_rationales'],
                                      'data_source' : cur_row['source']}
    
    
        cur_all_abbr[str(cur_abr)]['abr_whole_start'] = -1
        cur_all_abbr[str(cur_abr)]['abr_whole_end'] = -1
        # if short and long form of abbraviation exists
        if((cur_abr.end_char - cur_abr.start_char > 0) & (cur_abr._.long_form.end_char - cur_abr._.long_form.start_char > 0)):
            # Aaa Bbb Rrr (ABR) or ABR (Aaa Bbb Rrr)  
            if (cur_abr.start_char - cur_abr._.long_form.end_char   < 4 )| (cur_abr._.long_form.start_char - cur_abr.end_char < 4):
                cur_all_abbr[str(cur_abr)]['abr_whole_start'] = min(cur_abr.start_char, cur_abr._.long_form.start_char)
                cur_all_abbr[str(cur_abr)]['abr_whole_end'] = max(cur_abr.end_char, cur_abr._.long_form.end_char)
                
                
    for cur_abr in doc_jn._.abbreviations:
        list_jn.add(str(cur_abr))
        cur_all_abbr[str(cur_abr)] = {'abrv_text': str(cur_abr), 'abr_definition' : str(cur_abr._.long_form), 
                                      'abr_model' : 'en_ner_jnlpba_md', 'claim' : cur_claim, 'sf_start_char' : cur_abr.start_char, 'sf_end_char' : cur_abr.end_char,
                                     'lf_start_char' : cur_abr._.long_form.start_char, 'lf_end_char' : cur_abr._.long_form.end_char, 
                                     'org_label' : cur_row['label'], 'list_rationales' : cur_row['list_rationales'],
                                      'data_source' : cur_row['source']}
        
        
        cur_all_abbr[str(cur_abr)]['abr_whole_start'] = -1
        cur_all_abbr[str(cur_abr)]['abr_whole_end'] = -1
        # if short and long form of abbraviation exists
        if((cur_abr.end_char - cur_abr.start_char > 0) & (cur_abr._.long_form.end_char - cur_abr._.long_form.start_char > 0)):
            # Aaa Bbb Rrr (ABR) or ABR (Aaa Bbb Rrr)  
            if (cur_abr.start_char - cur_abr._.long_form.end_char   < 4 )| (cur_abr._.long_form.start_char - cur_abr.end_char < 4):
                cur_all_abbr[str(cur_abr)]['abr_whole_start'] = min(cur_abr.start_char, cur_abr._.long_form.start_char)
                cur_all_abbr[str(cur_abr)]['abr_whole_end'] = max(cur_abr.end_char, cur_abr._.long_form.end_char)+1        
    
    
    all_sentence_abbr.extend(list(cur_all_abbr.values()))
    
# en_core_sci_scibert and   en_ner_jnlpba_md having same abbrs

100%|██████████| 773/773 [01:14<00:00, 10.36it/s]


In [24]:
all_sentence_abbr

[{'abrv_text': 'CKD',
  'abr_definition': 'chronic kidney disease',
  'abr_model': 'en_ner_jnlpba_md',
  'claim': '40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.',
  'sf_start_char': 104,
  'sf_end_char': 107,
  'lf_start_char': 80,
  'lf_end_char': 102,
  'org_label': 'SUPPORTS',
  'list_rationales': ['CONCLUSION Treatment with high doses of folic acid and B vitamins did not improve survival or reduce the incidence of vascular disease in patients with advanced chronic kidney disease or end-stage renal disease.   \n'],
  'data_source': 'train',
  'abr_whole_start': 80,
  'abr_whole_end': 108},
 {'abrv_text': 'Th2',
  'abr_definition': 'T helper 2 cell',
  'abr_model': 'en_ner_jnlpba_md',
  'claim': 'A T helper 2 cell (Th2) environment impedes disease development in patients with systemic lupus erythematosus (SLE).',
  'sf_start_char': 19,
  'sf_end_char': 22,
  'lf_start_char': 2,
  'lf_end_char': 17,
  'org_lab

In [16]:
df_sentence_word_unq_abr = pd.DataFrame(all_sentence_abbr)

In [17]:
df_sentence_word_unq_abr

,abrv_text,abr_definition,abr_model,claim,sf_start_char,sf_end_char,lf_start_char,lf_end_char,org_label,list_rationales,data_source,abr_whole_start,abr_whole_end
0,CKD,chronic kidney disease,en_ner_jnlpba_md,40mg/day dosage of folic acid and 2mg/day dosa...,104,107,80,102,SUPPORTS,[CONCLUSION Treatment with high doses of folic...,train,80,108
1,Th2,T helper 2 cell,en_ner_jnlpba_md,A T helper 2 cell (Th2) environment impedes di...,19,22,2,17,REFUTES,"[Thus, in Lyn(-/-) mice, basophils and IgE aut...",train,2,23
2,SLE,systemic lupus erythematosus,en_ner_jnlpba_md,A T helper 2 cell (Th2) environment impedes di...,111,114,81,109,REFUTES,"[Thus, in Lyn(-/-) mice, basophils and IgE aut...",train,81,115
3,AMPK,AMP-activated protein kinase,en_ner_jnlpba_md,AMP-activated protein kinase (AMPK) activation...,30,34,0,28,SUPPORTS,[These studies implicate deficient AMPK activa...,train,0,35
4,ESCs,embryonic stem cells,en_ner_jnlpba_md,Androgenetic haploid mouse embryonic stem cell...,49,53,27,47,SUPPORTS,[Our results demonstrate that AG-haESCs can be...,train,27,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,IgG,Immunoglobulin G,en_ner_jnlpba_md,Human T-lymphotropic virus type-I-associated m...,132,135,114,130,SUPPORTS,[Antibodies to hnRNP-A1 cross-reacted with HTL...,dev,114,136
80,NSC,Neural Stem Cell,en_ner_jnlpba_md,MicroRNA is involved in the regulation of Neur...,60,63,42,58,SUPPORTS,[High levels of miR-184 promoted proliferation...,dev,42,64
81,NETs,Neutrophil extracellular traps,en_ner_jnlpba_md,Neutrophil extracellular traps (NETs) are rele...,32,36,0,30,SUPPORTS,"[Here we show that chromatin fibers, so-called...",dev,0,37
82,E. coli,Escherichia coli,en_ner_jnlpba_md,The tip of the inner tube of the toxic type VI...,115,122,97,113,SUPPORTS,[Our results indicate a new model of the T6SS ...,dev,97,123


In [18]:
df_sentence_word_unq_abr['abr_model'].value_counts()

en_ner_jnlpba_md    84
Name: abr_model, dtype: int64

## NER

In [19]:
from pattern.en import quantify, numerals, number,reflect

from nltk.corpus import wordnet as wn_nltk
from itertools import chain

from pattern.en import NOUN, VERB, ADJECTIVE, ADVERB, NOUN, VERB, ADJ, ADV, PRON, DET, PREP, ADP, NUM, CONJ, INTJ, PRT, PUNC, X
from pattern.en import  POS, CHUNK, INFINITIVE, PRESENT, PAST, FUTURE, FIRST, SECOND, THIRD,SINGULAR, PLURAL, SG, PL, PROGRESSIVE, PARTICIPLE, WORD
from pattern.text import UNIVERSAL
from pattern.en import conjugate, lemma, lexeme,PRESENT,SG, verbs, PARTICIPLE, ngrams
from pattern.en import wordnet as wn_pattern

from pattern.en import tree
from pattern.text.tree import Word
from pattern.en import parse, parsetree, tag, pprint

In [38]:
df_claim_evid_label.shape

(773, 4)

In [20]:
all_sentence_word_ner = []
for cur_index, cur_row in tqdm(df_claim_evid_label.iterrows(), total = len(df_claim_evid_label)):
#     doc_sci_brt = nlp_sci_brt(txt)
#     doc_sci_lg = nlp_sci_lg(txt)

    cur_claim = cur_row['claim']
    doc_cr = nlp_cr(cur_claim)
    doc_bc = nlp_bc(cur_claim)
    doc_bi = nlp_bi(cur_claim)
    doc_jn = nlp_jn(cur_claim)
    
    

    
#         all_tags.append({'ner_text': cur_ent.text, 'ner_label' : cur_ent.label_, 'ner_model' : 'en_core_sci_scibert', 'claim' : txt})
    
#     for cur_ent in doc_sci_lg.ents:
#         all_tags.append({'ner_text': cur_ent.text, 'ner_label' : cur_ent.label_, 'ner_model' : 'en_core_sci_lg', 'claim' : txt})

    
    cur_all_tags = {}
    
#     for cur_ent in doc_sci_brt.ents:
#         cur_all_tags[cur_ent.text] = {'ner_text': cur_ent.text, 'ner_label' : cur_ent.label_, 'ner_model' : 'en_core_sci_scibert', 'claim' : txt, 'start_char' : cur_ent.start_char, 'end_char' : cur_ent.end_char}

    for cur_ent in doc_cr.ents:      
        
        cur_all_tags[cur_ent.text] = {'ner_text': cur_ent.text, 'ner_label' : cur_ent.label_, 
                                      'ner_model' : 'en_ner_craft_md', 'claim' : cur_claim, 'start_char' : cur_ent.start_char, 
                                      'end_char' : cur_ent.end_char, 'org_label' : cur_row['label'], 'list_rationales' : cur_row['list_rationales'],
                                      'data_source' : cur_row['source']}
    
    for cur_ent in doc_bc.ents:
        cur_all_tags[cur_ent.text] = {'ner_text': cur_ent.text, 'ner_label' : cur_ent.label_, 
                                      'ner_model' : 'en_ner_bc5cdr_md', 'claim' : cur_claim, 'start_char' : cur_ent.start_char, 
                                      'end_char' : cur_ent.end_char, 'org_label' : cur_row['label'], 'list_rationales' : cur_row['list_rationales'],
                                      'data_source' : cur_row['source']}
        
    for cur_ent in doc_bi.ents:
        cur_all_tags[cur_ent.text] = {'ner_text': cur_ent.text, 'ner_label' : cur_ent.label_, 
                                      'ner_model' : 'en_ner_bionlp13cg_md', 'claim' : cur_claim, 'start_char' : cur_ent.start_char, 
                                      'end_char' : cur_ent.end_char, 'org_label' : cur_row['label'], 'list_rationales' : cur_row['list_rationales'],
                                      'data_source' : cur_row['source']}
    
    for cur_ent in doc_jn.ents:
        cur_all_tags[cur_ent.text] = {'ner_text': cur_ent.text, 'ner_label' : cur_ent.label_, 
                                      'ner_model' : 'en_ner_jnlpba_md', 'claim' : cur_claim, 'start_char' : cur_ent.start_char, 
                                      'end_char' : cur_ent.end_char, 'org_label' : cur_row['label'], 'list_rationales' : cur_row['list_rationales'],
                                      'data_source' : cur_row['source']}
    
    
    all_sentence_word_ner.extend(list(cur_all_tags.values()))

100%|██████████| 773/773 [00:22<00:00, 33.82it/s]


In [21]:
df_sentence_word_unq_ner = pd.DataFrame(all_sentence_word_ner)

In [22]:
df_sentence_word_unq_abr_ext = df_sentence_word_unq_abr.copy()

In [23]:
df_sentence_word_unq_abr_ext['comment'] = 'abbreviation'
df_sentence_word_unq_abr_ext['whole_text'] = df_sentence_word_unq_abr_ext['abr_definition'] + " ("+ df_sentence_word_unq_abr_ext['abrv_text'] + ")"

df_sentence_word_unq_abr_ext.rename(columns = {'whole_text' : 'ner_text', 'comment' : 'ner_label', 'abr_model' :'ner_model', 
                                                'claim' : 'claim', 'abr_whole_start' : 'start_char', 'abr_whole_end' : 'end_char'}, inplace= True)

In [24]:
df_sentence_word_unq_ner_abr = df_sentence_word_unq_ner.append(df_sentence_word_unq_abr_ext[['ner_label', 'ner_text', 'ner_model', 
                                                'claim', 'start_char', 'end_char', 'org_label', 'list_rationales', 'data_source']], ignore_index= True)

df_sentence_word_unq_ner_abr.sort_values(by = ['claim', 'start_char', 'end_char'], inplace= True)

df_sentence_word_unq_ner_abr.reset_index(drop= True, inplace=True)


In [25]:
HTML(df_sentence_word_unq_ner_abr.loc[9:19, :].sort_values(by = ['claim']).to_html())

,ner_text,ner_label,ner_model,claim,start_char,end_char,org_label,list_rationales,data_source
9,methadone,SIMPLE_CHEMICAL,en_ner_bionlp13cg_md,32% of liver transplantation programs required patients to discontinue methadone treatment in 2001.,71,80,SUPPORTS,[Policies requiring discontinuation of methadone in 32% of all programs contradict the evidence base for efficacy of long-term replacement therapies and potentially result in relapse of previously stable patients.],train
10,40mg/day,SIMPLE_CHEMICAL,en_ner_bionlp13cg_md,40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.,0,8,SUPPORTS,[CONCLUSION Treatment with high doses of folic acid and B vitamins did not improve survival or reduce the incidence of vascular disease in patients with advanced chronic kidney disease or end-stage renal disease. \n],train
11,folic acid,SIMPLE_CHEMICAL,en_ner_bionlp13cg_md,40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.,19,29,SUPPORTS,[CONCLUSION Treatment with high doses of folic acid and B vitamins did not improve survival or reduce the incidence of vascular disease in patients with advanced chronic kidney disease or end-stage renal disease. \n],train
12,2mg/day,CHEMICAL,en_ner_bc5cdr_md,40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.,34,41,SUPPORTS,[CONCLUSION Treatment with high doses of folic acid and B vitamins did not improve survival or reduce the incidence of vascular disease in patients with advanced chronic kidney disease or end-stage renal disease. \n],train
13,vitamin,CHEBI,en_ner_craft_md,40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.,52,59,SUPPORTS,[CONCLUSION Treatment with high doses of folic acid and B vitamins did not improve survival or reduce the incidence of vascular disease in patients with advanced chronic kidney disease or end-stage renal disease. \n],train
14,vitamin B12,SIMPLE_CHEMICAL,en_ner_bionlp13cg_md,40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.,52,63,SUPPORTS,[CONCLUSION Treatment with high doses of folic acid and B vitamins did not improve survival or reduce the incidence of vascular disease in patients with advanced chronic kidney disease or end-stage renal disease. \n],train
15,chronic kidney disease,DISEASE,en_ner_bc5cdr_md,40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.,80,102,SUPPORTS,[CONCLUSION Treatment with high doses of folic acid and B vitamins did not improve survival or reduce the incidence of vascular disease in patients with advanced chronic kidney disease or end-stage renal disease. \n],train
16,chronic kidney disease (CKD),abbreviation,en_ner_jnlpba_md,40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.,80,108,SUPPORTS,[CONCLUSION Treatment with high doses of folic acid and B vitamins did not improve survival or reduce the incidence of vascular disease in patients with advanced chronic kidney disease or end-stage renal disease. \n],train
17,kidney,ORGAN,en_ner_bionlp13cg_md,40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.,88,94,SUPPORTS,[CONCLUSION Treatment with high doses of folic acid and B vitamins did not improve survival or reduce the incidence of vascular disease in patients with advanced chronic kidney disease or end-stage renal disease. \n],train
18,CKD,CANCER,en_ner_bionlp13cg_md,40mg/day dosage of folic acid and 2mg/day dosage of vitamin B12 does not affect chronic kidney disease (CKD) progression.,104,107,SUPPORTS,[CONCLUSION Treatment with high doses of folic acid and B vitamins did not improve survival or 

In [26]:
def get_overlaped_span(start_1, end_1, start_2, end_2):
    if( max(end_1, end_2) - min(start_1, start_2) <= (end_1 - start_1) + (end_2 - start_2)):
        return (min(start_1, start_2) , max(end_1, end_2))
    else :
        return None

In [27]:
df_grp_sentence_word_unq_ner = df_sentence_word_unq_ner_abr.loc[:, :].groupby('claim')

In [28]:
df_sentence_word_unq_ner_abr_filtered = pd.DataFrame(columns=['ner_text', 'ner_label', 'ner_model', 
                                                'claim', 'start_char', 'end_char'])

for cur_claim, cur_df_group in tqdm(df_grp_sentence_word_unq_ner):
    cur_df_group.sort_values(by = ['start_char', 'end_char'], inplace= True)
    counter_index_outer = 0
    while counter_index_outer < len(cur_df_group):
        cur_span_start = cur_df_group['start_char']
        counter_index_inner = counter_index_outer+1
        while counter_index_inner < len(cur_df_group):  
            overlapped_span = get_overlaped_span(cur_df_group['start_char'].iloc[counter_index_outer], cur_df_group['end_char'].iloc[counter_index_outer],
                                 cur_df_group['start_char'].iloc[counter_index_inner], cur_df_group['end_char'].iloc[counter_index_inner])
            if overlapped_span != None:
                cur_df_group.loc[cur_df_group.index[counter_index_outer], 'start_char'] = overlapped_span[0]
                cur_df_group.loc[cur_df_group.index[counter_index_outer], 'end_char'] = overlapped_span[1]
                cur_df_group.drop(cur_df_group.index[[counter_index_inner]], inplace = True)
            else:
                counter_index_inner += 1 
        counter_index_outer += 1
    df_sentence_word_unq_ner_abr_filtered = df_sentence_word_unq_ner_abr_filtered.append(cur_df_group, ignore_index= True)


  0%|          | 0/668 [00:00<?, ?it/s]/home/qudratealahyratu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

In [29]:
df_grp_sentence_word_unq_ner_abr_filtered = df_sentence_word_unq_ner_abr_filtered.loc[:, :].groupby('claim')

In [30]:
df_sentence_word_unq_ner_abr_filtered.to_pickle(loc_df_scispacy_sentence_word_unq_ner_abr_filtered)

In [31]:
dic_map_strpos_patternpos = {'NOUN' :NOUN, 'VERB' : VERB, 'ADJECTIVE': ADJECTIVE, 'ADVERB' : ADVERB}

In [32]:
css_style = """
        <style>
        .org_claim_container {
          background-color: #f0f6fc;
          color: #000000cc;
          border: 2px ridge #72aee655;
          border-radius: 10px;
          margin: 5px;
          padding: 10px;
        }

        .term_container{
          background-color: #faebf6;
          color: #af0000dc;
          border-radius: 5px;
          border: 2px ridge #faebf613;
          font-size: 12pt;
          font-weight: bold;
        }

        .non_term_container{
          background-color: #fff7fd;
          color: #001630;
          font-size: 12pt;
        }   
        
        .meta_info_token{
        display: block;
        background-color: #00a6f309;
        border: 1px ridge #00a6f333;
        margine : 5px;
        padding : 5px;
        font-weight: 400;
        text-shadow: 0.2px 0.2px #00000055;
        box-shadow: 1px 1px #00660022;
        }
        </style>
    """

gen_html = css_style

list_dic_of_sent_tag_info = []

for cur_claim, cur_df_group in tqdm(df_grp_sentence_word_unq_ner_abr_filtered):
    doc_sci_brt = nlp_sci_brt(cur_claim)
    list_term_ranges = cur_df_group[['start_char', 'end_char']].values
    gen_html += '<div class="org_claim_container">'
    
    
    
    previous_term_end = 0
    for cur_indx, cur_term_range in enumerate(list_term_ranges):
        
        gen_html += '<span class = "non_term_container">'+ cur_claim[previous_term_end:cur_term_range[0]]+'</span>'
        gen_html += '<span class = "term_container">'+ cur_claim[cur_term_range[0]:cur_term_range[1]]+'</span>'
        
        doc_sci_brt = nlp_sci_brt(cur_claim[previous_term_end:cur_term_range[0]])
        
        
        for cur_token in doc_sci_brt:
            dic_token_all_info = {}
#             gen_html += '<span class = "meta_info_token">'+ str({'<b>token_text': cur_token.text, '</b>token_lemma' : cur_token.lemma_, 
#                                                                  'pos' : cur_token.pos_,'dep' : cur_token.dep_, 
#                                                                  'tag' : cur_token.tag_,
#                                                                  'tag_explain' : spacy.explain(cur_token.tag_),
#                                                                 })+'</span>'
            
            
            dic_token_all_info['claim'] = cur_claim, 
            dic_token_all_info['token'] = cur_token.text 
            
#             print('org label ', set(cur_df_group['org_label']))
            dic_token_all_info['org_label'] = list(set(cur_df_group['org_label']))[0]
            dic_token_all_info['list_rationales'] = list(cur_df_group['list_rationales'])[0]
            dic_token_all_info['data_source'] = list(set(cur_df_group['data_source']))[0]
#             dic_token_all_info['claim'] = 
            
#             print('\n\n#### spacy ####')
            
#             print({'token_text': cur_token.text, 'token_lemma' : cur_token.lemma_, 
#                                                                  'pos' : cur_token.pos_,'dep' : cur_token.dep_, 
#                                                                  'tag' : cur_token.tag_,
#                                                                  'tag_explain' : spacy.explain(cur_token.tag_),                                                                 
#                                                                 })
            
            
            dic_token_all_info['token_text_spacy'] = cur_token.text 
            dic_token_all_info['token_lemma_spacy'] = cur_token.lemma_ 
            dic_token_all_info['pos_spacy'] = cur_token.pos_
            dic_token_all_info['dep_spacy'] = cur_token.dep_
            dic_token_all_info['tag_spacy'] = cur_token.tag_
            dic_token_all_info['tag_explain_spacy'] = spacy.explain(cur_token.tag_),                                                                 
                                                                
#             print('\n#### NLTK ####')
            syns_wn_nltk = wn_nltk.synsets(cur_token.text)
            synset_wn_nltk = list(set(chain.from_iterable([word.lemma_names() for word in syns_wn_nltk])))
#             print('nltk syn : ', cur_token.text, ' >>> ', synset_wn_nltk)
            dic_token_all_info['wn_nltk_syn'] = synset_wn_nltk
            
            
#             print('morph :', cur_token.morph.to_dict())
            dic_token_all_info['morf_nltk'] = cur_token.morph.to_dict()
            
#             print('\n#### Pattern ####')
            pattern_syns_list = []
            
            try:
                pattern_syns_list.append(numerals(cur_token.text))
            except :
                print()
#             word_pattern = Word(cur_claim, cur_token.text)
#             print('word_pattern > ', word_pattern.string)


#             print('lemma', ' > ', lemma(cur_token.text))
            
            
            if(cur_token.pos_ in ['NOUN', 'VERB', 'ADJECTIVE', 'ADVERB'] ):
                cur_pos = str(cur_token.pos_).upper()
#                 print("cur_token > ", cur_token.text)
                
                for synset in wn_pattern.synsets(cur_token.text, pos=dic_map_strpos_patternpos[cur_token.pos_]):
#                     print("Token pos : ", synset.pos)
#                     print("Token similar : ", synset.similar())
#                     print("synonyms : %s" % synset.synonyms)      
#                     print("pos  : %s" % synset.lexname )    
                    pattern_syns_list += synset.synonyms
            pattern_syns_list = list(set(pattern_syns_list))
            dic_token_all_info['wn_pattern_syn'] = pattern_syns_list
#             print('pattern synonyms :', pattern_syns_list)
            
            list_dic_of_sent_tag_info.append(dic_token_all_info)
        previous_term_end = cur_term_range[1]
        
    gen_html += '<span class = "non_term_container">'+cur_claim[previous_term_end:]+'</span>'
    gen_html += '</div>'
    

    


  0%|          | 1/668 [00:00<02:36,  4.27it/s]

  0%|          | 2/668 [00:00<02:36,  4.25it/s]

  0%|          | 3/668 [00:01<05:00,  2.21it/s]

  1%|          | 4/668 [00:01<04:19,  2.55it/s]

  1%|          | 5/668 [00:02<04:12,  2.62it/s]

...

 99%|█████████▉| 662/668 [02:02<00:00,  7.08it/s]

 99%|█████████▉| 664/668 [02:03<00:00,  7.59it/s]

100%|█████████▉| 666/668 [02:03<00:00,  6.85it/s]

100%|█████████▉| 667/668 [02:03<00:00,  5.62it/s]

100%|██████████| 668/668 [02:03<00:00,  5.40it/s]

In [1]:
HTML(gen_html)

NameError: name 'HTML' is not defined

## POS

In [43]:
df_non_term_toekn_info.iloc[:10,:]

,claim,token,org_label,list_rationales,data_source,token_text_spacy,token_lemma_spacy,pos_spacy,dep_spacy,tag_spacy,tag_explain_spacy,wn_nltk_syn,morf_nltk,wn_pattern_syn
0,(1 in 5 million in UK have abnormal PrP positi...,1,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,1,1,NUM,compound,CD,"(cardinal number,)","[single, ane, i, I, one, unity, ace, 1]",{'NumType': 'Card'},[one]
1,(1 in 5 million in UK have abnormal PrP positi...,in,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,in,in,ADP,quantmod,IN,"(conjunction, subordinating or preposition,)","[atomic_number_49, In, inwards, inward, inch, ...",{},[]
2,(1 in 5 million in UK have abnormal PrP positi...,5,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,5,5,NUM,compound,CD,"(cardinal number,)","[V, five, 5, quintet, fivesome, pentad, quint,...",{'NumType': 'Card'},[five]
3,(1 in 5 million in UK have abnormal PrP positi...,million,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,million,million,NUM,nmod:npmod,CD,"(cardinal number,)","[one_thousand_thousand, trillion, zillion, bil...",{'NumType': 'Card'},[]
4,(1 in 5 million in UK have abnormal PrP positi...,in,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,in,in,ADP,ROOT,IN,"(conjunction, subordinating or preposition,)","[atomic_number_49, In, inwards, inward, inch, ...",{},[]
5,(1 in 5 million in UK have abnormal PrP positi...,,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,,,SPACE,dep,_SP,"(None,)",[],{'PunctType': 'Peri'},[]
6,(1 in 5 million in UK have abnormal PrP positi...,have,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,have,have,VERB,ROOT,VB,"(verb, base form,)","[own, stimulate, get, give, sustain, wealthy_p...",{'VerbForm': 'Inf'},"[own, stimulate, get, give, sustain, experienc..."
7,(1 in 5 million in UK have abnormal PrP positi...,abnormal,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,abnormal,abnormal,ADJ,xcomp,JJ,"(adjective,)","[abnormal, unnatural]",{'Degree': 'Pos'},[]
8,"(1,000 genomes project enables mapping of gene...","1,000",SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev,"1,000","1,000",NUM,ROOT,CD,"(cardinal number,)",[],{'NumType': 'Card'},[]
9,"(1,000 genomes project enables mapping of gene...",,SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev,,,SPACE,dep,_SP,"(None,)",[],{'PunctType': 'Peri'},[]


In [40]:
HTML(df_non_term_toekn_info[['claim', 'token', 'token_text_spacy', 'pos_spacy', 'tag_spacy', 'tag_explain_spacy']].to_html())

,claim,token,token_text_spacy,pos_spacy,tag_spacy,tag_explain_spacy
0,"(1 in 5 million in UK have abnormal PrP positivity.,)",1,1,NUM,CD,"(cardinal number,)"
1,"(1 in 5 million in UK have abnormal PrP positivity.,)",in,in,ADP,IN,"(conjunction, subordinating or preposition,)"
2,"(1 in 5 million in UK have abnormal PrP positivity.,)",5,5,NUM,CD,"(cardinal number,)"
3,"(1 in 5 million in UK have abnormal PrP positivity.,)",million,million,NUM,CD,"(cardinal number,)"
4,"(1 in 5 million in UK have abnormal PrP positivity.,)",in,in,ADP,IN,"(conjunction, subordinating or preposition,)"
5,"(1 in 5 million in UK have abnormal PrP positivity.,)",,,SPACE,_SP,"(None,)"
6,"(1 in 5 million in UK have abnormal PrP positivity.,)",have,have,VERB,VB,"(verb, base form,)"
7,"(1 in 5 million in UK have abnormal PrP positivity.,)",abnormal,abnormal,ADJ,JJ,"(adjective,)"
8,"(1,000 genomes project enables mapping of genetic sequence variation consisting of rare variants with larger penetrance effects than common variants.,)","1,000","1,000",NUM,CD,"(cardinal number,)"
9,"(1,000 genomes project enables mapping of genetic sequence variation consisting of rare variants with larger penetrance effects than common variants.,)",,,SPACE,_SP,"(None,)"


In [32]:
df_non_term_toekn_info = pd.DataFrame(list_dic_of_sent_tag_info)

In [33]:
df_non_term_toekn_info['pos_spacy'].value_counts()

NOUN     1070
ADP      1048
SPACE     979
VERB      868
ADJ       498
DET       407
PUNCT     106
ADV        84
CCONJ      80
PART       78
NUM        45
SCONJ      34
PROPN      27
PRON       22
AUX        17
X           9
SYM         3
Name: pos_spacy, dtype: int64

In [34]:
df_non_term_toekn_info['tag_spacy'].value_counts()

IN       1081
_SP       979
NN        851
JJ        446
DT        399
VBZ       338
NNS       219
VBN       183
VB        139
VBG        98
VBP        97
CC         80
RB         78
-RRB-      48
JJR        46
CD         45
TO         40
,          39
NNP        27
RBR        19
MD         17
WRB        16
VBD        13
-LRB-      10
FW          9
EX          8
WDT         8
PRP$        7
JJS         6
POS         6
WP          5
''          4
RBS         3
``          2
$           2
PRP         2
HYPH        2
RP          1
SYM         1
NFP         1
Name: tag_spacy, dtype: int64

In [35]:
df_non_term_toekn_info.groupby(['pos_spacy', 'tag_spacy']).size()

pos_spacy  tag_spacy
ADJ        JJ            446
           JJR            46
           JJS             6
ADP        IN           1047
           RP              1
ADV        RB             46
           RBR            19
           RBS             3
           WRB            16
AUX        MD             17
CCONJ      CC             80
DET        DT            399
           WDT             8
NOUN       NN            851
           NNS           219
NUM        CD             45
PART       POS             6
           RB             32
           TO             40
PRON       EX              8
           PRP             2
           PRP$            7
           WP              5
PROPN      NNP            27
PUNCT      ''              4
           ,              39
           -LRB-          10
           -RRB-          48
           HYPH            2
           NFP             1
           ``              2
SCONJ      IN             34
SPACE      _SP           979
SYM        $          

## Debug

In [ ]:
!conda env list

In [ ]:
!/home/computeruser/anaconda3/envs/scifact/bin/pip install pattern

In [ ]:
!/home/computeruser/anaconda3/envs/scifact/bin/pip install --no-binary :all: nmslib

In [ ]:
!conda install pattern